In [2]:
from typing import LiteralString
import os
import duckdb

def process(sql: LiteralString, file: str) -> None:
    table = duckdb.sql(sql)
    table.to_parquet(f"./tmp/{file}.parquet")
    print(f"{file} - {int(os.path.getsize(f"./tmp/{file}.parquet") / (1024 * 1024))}")
    table.show()

In [3]:
process("""
SELECT
    region,
    date,
    url[length('https://open.spotify.com/track/')+1:] as track_id,
    streams
FROM '../../data/charts.csv'
WHERE chart = 'top200'
""", "charts_fmt")

charts_fmt - 381
┌───────────┬────────────┬────────────────────────┬─────────┐
│  region   │    date    │        track_id        │ streams │
│  varchar  │    date    │        varchar         │  int64  │
├───────────┼────────────┼────────────────────────┼─────────┤
│ Argentina │ 2017-01-01 │ 6mICuAdrwEjh6Y6lroV2Kg │  253019 │
│ Argentina │ 2017-01-01 │ 7DM4BPaS7uofFul3ywMe46 │  223988 │
│ Argentina │ 2017-01-01 │ 3AEZUABDXNtecAOSC1qTfo │  210943 │
│ Argentina │ 2017-01-01 │ 6rQSrBHf7HlZjtcMZ4S4bO │  173865 │
│ Argentina │ 2017-01-01 │ 58IL315gMSTD37DOZPJ2hf │  153956 │
│ Argentina │ 2017-01-01 │ 5J1c3M4EldCfNxXwrwt8mT │  151140 │
│ Argentina │ 2017-01-01 │ 1MpKZi1zTXpERKwxmOu1PH │  148369 │
│ Argentina │ 2017-01-01 │ 3QwBODjSEzelZyVjxPOHdq │  143004 │
│ Argentina │ 2017-01-01 │ 0sXvAOmXgjR2QUqLK1MltU │  126389 │
│ Argentina │ 2017-01-01 │ 20ZAJdsKB5IGbGj4ilRt2o │  112012 │
│   ·       │     ·      │           ·            │     ·   │
│   ·       │     ·      │           ·            │  

In [4]:
process("""
SELECT
    region,
    date,
    SUM(streams) as streams
FROM './tmp/charts_fmt.parquet'
GROUP BY region, date
ORDER BY region, date
""", "charts_daily_sum")

charts_daily_sum - 0
┌───────────┬────────────┬──────────┐
│  region   │    date    │ streams  │
│  varchar  │    date    │  int128  │
├───────────┼────────────┼──────────┤
│ Argentina │ 2017-01-01 │  7888872 │
│ Argentina │ 2017-01-02 │  6010041 │
│ Argentina │ 2017-01-03 │  5921907 │
│ Argentina │ 2017-01-04 │  6019573 │
│ Argentina │ 2017-01-05 │  6223646 │
│ Argentina │ 2017-01-06 │  6485179 │
│ Argentina │ 2017-01-07 │  6840811 │
│ Argentina │ 2017-01-08 │  5691182 │
│ Argentina │ 2017-01-09 │  5880184 │
│ Argentina │ 2017-01-10 │  5896533 │
│   ·       │     ·      │     ·    │
│   ·       │     ·      │     ·    │
│   ·       │     ·      │     ·    │
│ Brazil    │ 2019-07-13 │ 33582302 │
│ Brazil    │ 2019-07-14 │ 27615671 │
│ Brazil    │ 2019-07-15 │ 25856711 │
│ Brazil    │ 2019-07-16 │ 26449349 │
│ Brazil    │ 2019-07-17 │ 26636713 │
│ Brazil    │ 2019-07-18 │ 27154103 │
│ Brazil    │ 2019-07-19 │ 30321659 │
│ Brazil    │ 2019-07-20 │ 32104543 │
│ Brazil    │ 2019-07-21 │ 27

In [5]:
process("""
SELECT
    region,
    YEAR(date) as year,
    AVG(streams) as stream_avg,
    STDDEV(streams) as stream_dev
FROM './tmp/charts_daily_sum.parquet'
GROUP BY region, year
ORDER BY region, year
""", "charts_yearly_stats")

charts_yearly_stats - 0
┌───────────────┬───────┬────────────────────┬────────────────────┐
│    region     │ year  │     stream_avg     │     stream_dev     │
│    varchar    │ int64 │       double       │       double       │
├───────────────┼───────┼────────────────────┼────────────────────┤
│ Argentina     │  2017 │  7424229.113259668 │ 1034041.7027895374 │
│ Argentina     │  2018 │  8642794.898630137 │  971236.3782346837 │
│ Argentina     │  2019 │ 10852934.652054794 │ 1505170.5526055752 │
│ Argentina     │  2020 │ 11974701.893442623 │ 2195595.9299331396 │
│ Argentina     │  2021 │  14691098.39010989 │ 1835217.3342268008 │
│ Australia     │  2017 │  9070440.314917127 │ 1217620.9157923514 │
│ Australia     │  2018 │ 11125632.709589042 │ 1184823.8273182358 │
│ Australia     │  2019 │ 11930231.084931508 │  1209072.960792116 │
│ Australia     │  2020 │ 11986660.775956284 │ 1581691.8264637932 │
│ Australia     │  2021 │ 11450993.815864023 │  724796.2088324842 │
│     ·         │    ·  

In [6]:
process("""
SELECT
    region,
    date,
    CASE
        WHEN stream_std < -1.5 THEN 'VERY LOW'
        WHEN stream_std < -0.5 THEN 'LOW'
        WHEN stream_std > 1.5 THEN 'VERY HIGH'
        WHEN stream_std > 0.5 THEN 'HIGH'
        ELSE 'AVERAGE'
    END AS popularity
FROM (
    SELECT
        d.region,
        d.date,
        (d.streams - y.stream_avg) / y.stream_dev as stream_std
    FROM './tmp/charts_daily_sum.parquet' AS d
    JOIN './tmp/charts_yearly_stats.parquet' AS y
        ON YEAR(d.date) = y.year AND d.region = y.region
)
""", "charts_daily_popularity")

charts_daily_popularity - 0
┌───────────┬────────────┬────────────┐
│  region   │    date    │ popularity │
│  varchar  │    date    │  varchar   │
├───────────┼────────────┼────────────┤
│ Argentina │ 2017-01-01 │ AVERAGE    │
│ Argentina │ 2017-01-02 │ LOW        │
│ Argentina │ 2017-01-03 │ LOW        │
│ Argentina │ 2017-01-04 │ LOW        │
│ Argentina │ 2017-01-05 │ LOW        │
│ Argentina │ 2017-01-06 │ LOW        │
│ Argentina │ 2017-01-07 │ LOW        │
│ Argentina │ 2017-01-08 │ VERY LOW   │
│ Argentina │ 2017-01-09 │ LOW        │
│ Argentina │ 2017-01-10 │ LOW        │
│   ·       │     ·      │  ·         │
│   ·       │     ·      │  ·         │
│   ·       │     ·      │  ·         │
│ Brazil    │ 2018-04-26 │ AVERAGE    │
│ Brazil    │ 2018-04-27 │ HIGH       │
│ Brazil    │ 2018-04-28 │ VERY HIGH  │
│ Brazil    │ 2018-04-29 │ AVERAGE    │
│ Brazil    │ 2018-04-30 │ AVERAGE    │
│ Brazil    │ 2018-05-01 │ AVERAGE    │
│ Brazil    │ 2018-05-02 │ LOW        │
│ Brazil    

In [7]:
process("""
SELECT
    c.country,
    w.date,
    AVG(w.temperature_c) as temperature_c,
    COALESCE(AVG(w.precipitation_mm), 0) as precipitation_mm
FROM (
    SELECT
        station_id,
        date::DATE as date,
        avg_temp_c as temperature_c,
        precipitation_mm,
    FROM '../../data/daily_weather.parquet'
    WHERE date BETWEEN '2017-01-01' AND '2021-12-31'
) AS w
JOIN '../../data/cities.csv' AS c
    ON w.station_id = c.station_id
GROUP BY c.country, w.date
HAVING temperature_c IS NOT NULL
ORDER BY c.country, w.date
""", "daily_country_weather")

daily_country_weather - 2
┌─────────────┬────────────┬──────────────────────┬──────────────────┐
│   country   │    date    │    temperature_c     │ precipitation_mm │
│   varchar   │    date    │        double        │      double      │
├─────────────┼────────────┼──────────────────────┼──────────────────┤
│ Afghanistan │ 2017-01-01 │   5.3166666666666655 │              0.0 │
│ Afghanistan │ 2017-01-02 │    5.016666666666667 │              2.0 │
│ Afghanistan │ 2017-01-03 │   3.0666666666666664 │           10.725 │
│ Afghanistan │ 2017-01-04 │                 2.65 │            109.0 │
│ Afghanistan │ 2017-01-05 │   1.9333333333333333 │            29.95 │
│ Afghanistan │ 2017-01-06 │   0.9833333333333331 │             7.25 │
│ Afghanistan │ 2017-01-07 │   0.2833333333333334 │             33.0 │
│ Afghanistan │ 2017-01-08 │ -0.07999999999999999 │             18.0 │
│ Afghanistan │ 2017-01-09 │ -0.21666666666666665 │              0.0 │
│ Afghanistan │ 2017-01-10 │    0.866666666666667 │

In [8]:
process("""
WITH wdi AS (
    SELECT
        "Country Name" as country,
        "Indicator Code" as code,
        "2016", "2017", "2018", "2019", "2020", "2021"
    FROM read_csv('../../data/WDIData.csv', header=True)
    WHERE code IN ('SP.RUR.TOTL.ZS', 'SP.DYN.TFRT.IN', 'NY.GDP.PCAP.CD', 'IT.CEL.SETS.P2', 'SM.POP.REFG', 'SP.POP.TOTL')
)
SELECT
    country,
    year,
    rural_population_percent,
    fertility_rate,
    gdp_per_capita_usd,
    mobile_subscriptions_per_100,
    refugee_population / total_population * 1000 AS refugee_population_promille,
FROM (
    SELECT
        country, year,
        MAX(CASE WHEN code = 'SP.RUR.TOTL.ZS' THEN value END) AS rural_population_percent,
        MAX(CASE WHEN code = 'SP.DYN.TFRT.IN' THEN value END) AS fertility_rate,
        MAX(CASE WHEN code = 'NY.GDP.PCAP.CD' THEN value END) AS gdp_per_capita_usd,
        MAX(CASE WHEN code = 'IT.CEL.SETS.P2' THEN value END) AS mobile_subscriptions_per_100,
        MAX(CASE WHEN code = 'SM.POP.REFG' THEN value END) AS refugee_population,
        MAX(CASE WHEN code = 'SP.POP.TOTL' THEN value END) AS total_population
    FROM (
        SELECT country, code, 2016 AS year, "2017" AS value FROM wdi
        UNION ALL
        SELECT country, code, 2017 AS year, "2017" AS value FROM wdi
        UNION ALL
        SELECT country, code, 2018 AS year, "2018" AS value FROM wdi
        UNION ALL
        SELECT country, code, 2019 AS year, "2019" AS value FROM wdi
        UNION ALL
        SELECT country, code, 2020 AS year, "2020" AS value FROM wdi
        UNION ALL
        SELECT country, code, 2021 AS year, "2021" AS value FROM wdi
    )
    GROUP BY country, year
)
WHERE rural_population_percent IS NOT NULL
    AND fertility_rate IS NOT NULL
    AND gdp_per_capita_usd IS NOT NULL
    AND mobile_subscriptions_per_100 IS NOT NULL
    AND refugee_population IS NOT NULL
    AND total_population IS NOT NULL
ORDER BY country, year
""", "wdi_normalized")

wdi_normalized - 0
┌─────────────────────────────┬───────┬──────────────────────────┬──────────────────┬────────────────────┬──────────────────────────────┬─────────────────────────────┐
│           country           │ year  │ rural_population_percent │  fertility_rate  │ gdp_per_capita_usd │ mobile_subscriptions_per_100 │ refugee_population_promille │
│           varchar           │ int32 │          double          │      double      │       double       │            double            │           double            │
├─────────────────────────────┼───────┼──────────────────────────┼──────────────────┼────────────────────┼──────────────────────────────┼─────────────────────────────┤
│ Afghanistan                 │  2016 │                    74.75 │            5.129 │   530.149830802984 │                  67.13641492 │           2.130182913434396 │
│ Afghanistan                 │  2017 │                    74.75 │            5.129 │   530.149830802984 │                  67.13641492 │    

In [9]:
process("""
SELECT
    prv.country,
    d.date,
    prv.rural_population_percent * (1 - (EXTRACT('dayofyear' FROM d.date) / 365)) + nxt.rural_population_percent * (EXTRACT('dayofyear' FROM d.date) / 365) AS rural_population_percent,
    prv.fertility_rate * (1 - (EXTRACT('dayofyear' FROM d.date) / 365)) + nxt.fertility_rate * (EXTRACT('dayofyear' FROM d.date) / 365) AS fertility_rate,
    prv.gdp_per_capita_usd * (1 - (EXTRACT('dayofyear' FROM d.date) / 365)) + nxt.gdp_per_capita_usd * (EXTRACT('dayofyear' FROM d.date) / 365) AS gdp_per_capita_usd,
    prv.mobile_subscriptions_per_100 * (1 - (EXTRACT('dayofyear' FROM d.date) / 365)) + nxt.mobile_subscriptions_per_100 * (EXTRACT('dayofyear' FROM d.date) / 365) AS mobile_subscriptions_per_100,
    prv.refugee_population_promille * (1 - (EXTRACT('dayofyear' FROM d.date) / 365)) + nxt.refugee_population_promille * (EXTRACT('dayofyear' FROM d.date) / 365) AS refugee_population_promille
FROM (
    SELECT generate_series::DATE AS date
    FROM generate_series(DATE '2017-01-01', DATE '2021-12-31', INTERVAL '1' DAY)
) AS d
JOIN './tmp/wdi_normalized.parquet' AS nxt
    ON YEAR(d.date) = nxt.year
JOIN './tmp/wdi_normalized.parquet' AS prv
    ON YEAR(d.date) - 1 = prv.year AND prv.country = nxt.country
ORDER BY prv.country, d.date
""", "wdi_interpolated")

wdi_interpolated - 12
┌─────────────┬────────────┬──────────────────────────┬────────────────────┬────────────────────┬──────────────────────────────┬─────────────────────────────┐
│   country   │    date    │ rural_population_percent │   fertility_rate   │ gdp_per_capita_usd │ mobile_subscriptions_per_100 │ refugee_population_promille │
│   varchar   │    date    │          double          │       double       │       double       │            double            │           double            │
├─────────────┼────────────┼──────────────────────────┼────────────────────┼────────────────────┼──────────────────────────────┼─────────────────────────────┤
│ Afghanistan │ 2017-01-01 │                    74.75 │              5.129 │   530.149830802984 │            67.13641492000002 │           2.130182913434396 │
│ Afghanistan │ 2017-01-02 │                    74.75 │              5.129 │  530.1498308029841 │                  67.13641492 │           2.130182913434396 │
│ Afghanistan │ 2017-01-

In [10]:
process("""
SELECT track_id, UNNEST(artists) AS artist FROM (
    SELECT
        track_id,
        string_split(MAX(artists)[3:-3], ''', ''') AS artists
    FROM '../../data/api.csv'
    GROUP BY track_id
)
""", "api_track_to_artist")

api_track_to_artist - 4
┌────────────────────────┬─────────────────────┐
│        track_id        │       artist        │
│        varchar         │       varchar       │
├────────────────────────┼─────────────────────┤
│ 2l9XbXBbKoEfc1JdCvSz4n │ Achille Lauro       │
│ 2l9XbXBbKoEfc1JdCvSz4n │ Boss Doms           │
│ 2l9XbXBbKoEfc1JdCvSz4n │ Gemitaiz            │
│ 2l9XbXBbKoEfc1JdCvSz4n │ Quentin40           │
│ 2l9XbXBbKoEfc1JdCvSz4n │ Puritano            │
│ 4aIrVhmrJHTnklK2wRxK11 │ RAP VIỆT            │
│ 4aIrVhmrJHTnklK2wRxK11 │ Kellie              │
│ 7F5O2pMgNNjONTbdqQx9b3 │ Sidhu Moose Wala    │
│ 7F5O2pMgNNjONTbdqQx9b3 │ Morrisson           │
│ 5UnRXuem4qHseiCREBB7D7 │ Morad               │
│           ·            │   ·                 │
│           ·            │   ·                 │
│           ·            │   ·                 │
│ 1xitlGTkcFjDeDSvRLN0nF │ José Madero         │
│ 4bWdlExSeKAotB6uJwxFlf │ Jambao              │
│ 2Tz3KnoARGSMYIiH8KPwlz │ Koda              

In [11]:
process("""
SELECT track_id, UNNEST(genres) AS genre FROM (
    SELECT
        track_id,
        string_split(MAX(artist_genres)[3:-3], ''', ''') AS genres
    FROM '../../data/api.csv'
    GROUP BY track_id
)
""", "api_track_to_genre")

api_track_to_genre - 4
┌────────────────────────┬──────────────────┐
│        track_id        │      genre       │
│        varchar         │     varchar      │
├────────────────────────┼──────────────────┤
│ 0knsBJ21cqAbgjdr2i0R9b │ german dance     │
│ 0knsBJ21cqAbgjdr2i0R9b │ eurodance        │
│ 0knsBJ21cqAbgjdr2i0R9b │ partyschlager    │
│ 0knsBJ21cqAbgjdr2i0R9b │ hands up         │
│ 67sYutaZnLRZd6rRfECgJg │ greek drill      │
│ 3QzAOrNlsabgbMwlZt7TAY │ pop rap          │
│ 3QzAOrNlsabgbMwlZt7TAY │ southern hip hop │
│ 3QzAOrNlsabgbMwlZt7TAY │ pop              │
│ 3QzAOrNlsabgbMwlZt7TAY │ melodic rap      │
│ 3QzAOrNlsabgbMwlZt7TAY │ cali rap         │
│           ·            │    ·             │
│           ·            │    ·             │
│           ·            │    ·             │
│ 3gOwkbPvMOjzkavWbzKKZM │ pop urbaine      │
│ 3gOwkbPvMOjzkavWbzKKZM │ french hip hop   │
│ 3svle4JD98qjX8WVADVEMA │ norwegian indie  │
│ 3svle4JD98qjX8WVADVEMA │ norwegian pop    │
│ 0n2bvWvMw

In [12]:
process("""
SELECT
    c.region,
    c.date,
    MD5(a.artist) AS artist_id,
    c.streams
FROM './tmp/charts_fmt.parquet' AS c
JOIN './tmp/api_track_to_artist.parquet' AS a
    ON c.track_id = a.track_id
""", "charts_artists")

charts_artists - 497
┌─────────────┬────────────┬──────────────────────────────────┬─────────┐
│   region    │    date    │            artist_id             │ streams │
│   varchar   │    date    │             varchar              │  int64  │
├─────────────┼────────────┼──────────────────────────────────┼─────────┤
│ Netherlands │ 2017-01-05 │ ff51ada376d6ddc54bfae2f3b29736cd │   16090 │
│ Netherlands │ 2017-01-05 │ 6175b9f6699984f1ede4bea3a5c30fe1 │   16085 │
│ Netherlands │ 2017-01-05 │ 65ccd361b7db95e47b4c064042a19487 │   16052 │
│ Netherlands │ 2017-01-05 │ a619372dc055d1586d2209d3b5b2d0c1 │   16036 │
│ Netherlands │ 2017-01-05 │ 49d7cfbfaa2be24374e452d447bbc1e3 │   15849 │
│ Netherlands │ 2017-01-05 │ 35d30b93cd68c6b8fb34c1ab7f346e3c │   15459 │
│ Netherlands │ 2017-01-05 │ b150f254e79816bcde37b04180539289 │   15378 │
│ Netherlands │ 2017-01-05 │ 18f327c0eb1a48bc524d245f39200de0 │   15312 │
│ Netherlands │ 2017-01-05 │ 308cffe51978ee3729cdbf64a3631867 │   15081 │
│ Netherlands │ 2

In [13]:
process("""
SELECT
    c.region,
    c.date,
    t.genre,
    c.streams
FROM './tmp/charts_fmt.parquet' AS c
JOIN './tmp/api_track_to_genre.parquet' AS t
    ON c.track_id = t.track_id
""", "charts_genres")

charts_genres - 385
┌────────────────┬────────────┬─────────────────────────┬─────────┐
│     region     │    date    │          genre          │ streams │
│    varchar     │    date    │         varchar         │  int64  │
├────────────────┼────────────┼─────────────────────────┼─────────┤
│ France         │ 2018-02-05 │ trap latino             │   56509 │
│ France         │ 2018-02-05 │ uk pop                  │   55302 │
│ France         │ 2018-02-05 │ french hip hop          │   54797 │
│ France         │ 2018-02-05 │ old school rap francais │   53970 │
│ France         │ 2018-02-05 │ edm                     │   52859 │
│ France         │ 2018-02-05 │ rock                    │   52540 │
│ France         │ 2018-02-05 │ post-teen pop           │   52074 │
│ France         │ 2018-02-05 │ old school rap francais │   51916 │
│ France         │ 2018-02-05 │ uk pop                  │   51530 │
│ France         │ 2018-02-05 │ french pop              │   50620 │
│   ·            │     ·    

In [38]:
process("""
SELECT
    region,
    date,
    COALESCE(MAX(CASE WHEN genre = 'pop' THEN total_streams END), 0) AS pop,
    COALESCE(MAX(CASE WHEN genre = 'rap' THEN total_streams END), 0) AS rap,
    COALESCE(MAX(CASE WHEN genre = 'rock' THEN total_streams END), 0) AS rock,
    COALESCE(MAX(CASE WHEN genre = 'edm' THEN total_streams END), 0) AS edm,
    COALESCE(MAX(CASE WHEN genre = 'hip hop' THEN total_streams END), 0) AS hip_hop,
    COALESCE(MAX(CASE WHEN genre = 'trap latino' THEN total_streams END), 0) AS trap_latino,
    COALESCE(MAX(CASE WHEN genre = 'reggaeton' THEN total_streams END), 0) AS reggaeton,
    COALESCE(MAX(CASE WHEN genre = 'electropop' THEN total_streams END), 0) AS electropop,
    COALESCE(MAX(CASE WHEN genre = 'dance pop' THEN total_streams END), 0) AS dance_pop,
    COALESCE(MAX(CASE WHEN genre = 'pop rap' THEN total_streams END), 0) AS pop_rap,
    COALESCE(MAX(CASE WHEN genre = 'musica mexicana' THEN total_streams END), 0) AS musica_mexicana,
    COALESCE(MAX(CASE WHEN genre = 'trap' THEN total_streams END), 0) AS trap,
    COALESCE(MAX(CASE WHEN genre = 'modern rock' THEN total_streams END), 0) AS modern_rock,
    COALESCE(MAX(CASE WHEN genre = 'classic rock' THEN total_streams END), 0) AS classic_rock,
    COALESCE(MAX(CASE WHEN genre = 'uk pop' THEN total_streams END), 0) AS uk_pop,
    COALESCE(MAX(CASE WHEN genre = 'k-pop' THEN total_streams END), 0) AS k_pop,
    COALESCE(MAX(CASE WHEN genre = 'tropical house' THEN total_streams END), 0) AS tropical_house,
    COALESCE(MAX(CASE WHEN genre = 'melodic rap' THEN total_streams END), 0) AS melodic_rap,
    COALESCE(MAX(CASE WHEN genre = 'canadian pop' THEN total_streams END), 0) AS canadian_pop,
    COALESCE(MAX(CASE WHEN genre = 'modern bollywood' THEN total_streams END), 0) AS modern_bollywood
FROM (
    SELECT
        region,
        date,
        genre,
        SUM(streams) AS total_streams
    FROM './tmp/charts_genres.parquet'
    WHERE genre IN ('pop', 'rap', 'rock', 'edm', 'hip hop', 'trap latino', 'reggaeton', 'electropop', 'dance pop', 'pop rap', 'musica mexicana', 'trap', 'modern rock', 'classic rock', 'uk pop', 'k-pop', 'tropical house', 'melodic rap', 'canadian pop', 'modern bollywood')
    GROUP BY region, date, genre
)
GROUP BY region, date
ORDER BY region, date
""", "charts_daily_genres")

charts_daily_genres - 8
┌───────────┬────────────┬─────────┬────────┬────────┬────────┬─────────┬─────────────┬───────────┬────────────┬───────────┬─────────┬─────────────────┬────────┬─────────────┬──────────────┬─────────┬────────┬────────────────┬─────────────┬──────────────┬──────────────────┐
│  region   │    date    │   pop   │  rap   │  rock  │  edm   │ hip_hop │ trap_latino │ reggaeton │ electropop │ dance_pop │ pop_rap │ musica_mexicana │  trap  │ modern_rock │ classic_rock │ uk_pop  │ k_pop  │ tropical_house │ melodic_rap │ canadian_pop │ modern_bollywood │
│  varchar  │    date    │ int128  │ int128 │ int128 │ int128 │ int128  │   int128    │  int128   │   int128   │  int128   │ int128  │     int128      │ int128 │   int128    │    int128    │ int128  │ int128 │     int128     │   int128    │    int128    │      int128      │
├───────────┼────────────┼─────────┼────────┼────────┼────────┼─────────┼─────────────┼───────────┼────────────┼───────────┼─────────┼─────────────────

In [43]:
process("""
SELECT
    g.region,
    g.date,
    g.pop / s.streams AS pop,
    g.rap / s.streams AS rap,
    g.rock / s.streams AS rock,
    g.edm / s.streams AS edm,
    g.hip_hop / s.streams AS hip_hop,
    g.trap_latino / s.streams AS trap_latino,
    g.reggaeton / s.streams AS reggaeton,
    g.electropop / s.streams AS electropop,
    g.dance_pop / s.streams AS dance_pop,
    g.pop_rap / s.streams AS pop_rap,
    g.musica_mexicana / s.streams AS musica_mexicana,
    g.trap / s.streams AS trap,
    g.modern_rock / s.streams AS modern_rock,
    g.classic_rock / s.streams AS classic_rock,
    g.uk_pop / s.streams AS uk_pop,
    g.k_pop / s.streams AS k_pop,
    g.tropical_house / s.streams AS tropical_house,
    g.melodic_rap / s.streams AS melodic_rap,
    g.canadian_pop / s.streams AS canadian_pop,
    g.modern_bollywood / s.streams AS modern_bollywood
FROM './tmp/charts_daily_genres.parquet' AS g
JOIN './tmp/charts_daily_sum.parquet' AS s
    ON g.region = s.region AND g.date = s.date
""", "charts_genre_popularity")

charts_genre_popularity - 13
┌───────────┬────────────┬─────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬─────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬──────────────────────┬───────────────────────┬──────────────────────┬───────────────────────┬──────────────────────┬──────────────────┐
│  region   │    date    │         pop         │         rap          │         rock         │         edm          │       hip_hop        │     trap_latino      │      reggaeton       │      electropop      │      dance_pop       │       pop_rap        │ musica_mexicana │         trap          │      modern_rock      │     classic_rock      │        uk_pop        │         k_pop         │    tropical_house    │      melodic_rap      │     canadian_pop     │ modern_bollywood │
│  varc

In [51]:
process("""
SELECT
    cgp.region AS country,
    cgp.date,
    cds.streams AS daily_streams,
    cdp.popularity AS daily_popularity,
    dcw.temperature_c AS temperature_c,
    dcw.precipitation_mm AS precipitation_mm,
    wdi.rural_population_percent AS "wdi:rural_population_percent",
    wdi.fertility_rate AS "wdi:fertility_rate",
    wdi.gdp_per_capita_usd AS "wdi:gdp_per_capita_usd",
    wdi.mobile_subscriptions_per_100 AS "wdi:mobile_subscriptions_per_100",
    wdi.refugee_population_promille AS "wdi:refugee_population_promille",
    cgp.pop AS "genre:pop",
    cgp.rap AS "genre:rap",
    cgp.rock AS "genre:rock",
    cgp.edm AS "genre:edm",
    cgp.hip_hop AS "genre:hip_hop",
    cgp.trap_latino AS "genre:trap_latino",
    cgp.reggaeton AS "genre:reggaeton",
    cgp.electropop AS "genre:electropop",
    cgp.dance_pop AS "genre:dance_pop",
    cgp.pop_rap AS "genre:pop_rap",
    cgp.musica_mexicana AS "genre:musica_mexicana",
    cgp.trap AS "genre:trap",
    cgp.modern_rock AS "genre:modern_rock",
    cgp.classic_rock AS "genre:classic_rock",
    cgp.uk_pop AS "genre:uk_pop",
    cgp.k_pop AS "genre:k_pop",
    cgp.tropical_house AS "genre:tropical_house",
    cgp.melodic_rap AS "genre:melodic_rap",
    cgp.canadian_pop AS "genre:canadian_pop",
    cgp.modern_bollywood AS "genre:modern_bollywood",
FROM './tmp/charts_genre_popularity.parquet' AS cgp
JOIN './tmp/charts_daily_popularity.parquet' AS cdp
    ON cgp.region = cdp.region AND cgp.date = cdp.date
JOIN './tmp/charts_daily_sum.parquet' AS cds
    ON cgp.region = cds.region AND cgp.date = cds.date
JOIN './tmp/daily_country_weather.parquet' AS dcw
    ON cgp.region = dcw.country AND cgp.date = dcw.date
JOIN './tmp/wdi_interpolated.parquet' AS wdi
    ON cgp.region = wdi.country AND cgp.date = wdi.date
ORDER BY cgp.region, cgp.date
""", "output")

output - 15
┌───────────┬────────────┬───────────────┬──────────────────┬────────────────────┬────────────────────┬──────────────────────────────┬────────────────────┬────────────────────────┬──────────────────────────────────┬─────────────────────────────────┬─────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬──────────────────────┬──────────────────────┬───────────────────────┬──────────────────────┬───────────────────────┬──────────────────────┬────────────────────────┐
│  country  │    date    │ daily_streams │ daily_popularity │   temperature_c    │  precipitation_mm  │ wdi:rural_population_percent │ wdi:fertility_rate │ wdi:gdp_per_capita_usd │ wdi:mobile_subscriptions_per_100 │ wdi:refugee_population_promille │      genre:pop      │ 